In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

#https://huggingface.co/ckiplab/gpt2-base-chinese
model_name = "ckiplab/gpt2-base-chinese"

# 檢查是否有 GPU 可以用（如果有會快很多）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用裝置：{device}")

使用裝置：cuda


In [8]:
# 載入指定的語言模型與 tokenizer
# model_name 是你指定的模型名稱，例如 "ckiplab/gpt2-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name)  # 用於將文字轉換為模型輸入的 token
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)  # 載入因果語言模型並指定裝置（例如 CPU 或 GPU）

# 定義一個函式，用來根據提示詞（prompt）與選擇的風格生成文字
def generate_text(prompt, style="default", max_new_tokens=150, temperature=0.9, top_k=50, top_p=0.95):
    # 定義可選的風格標籤字典，對應不同的語氣或文體
    style_labels = {
        "default": "一般",
        "fairy_tale": "童話",
        "ancient_chinese": "古文",
        "child_style": "兒童",
        "mystery": "懸疑",
        "sci_fi": "科幻",
        "wuxia": "武俠",
        "diary": "日記",
        "news": "新聞",
        "poetic": "詩詞"
    }

    # 根據 style 參數取得對應的中文風格標籤，若未指定則預設為「一般」
    style_tag = style_labels.get(style, "一般")

    # 將風格標籤加入提示詞中，以引導模型產生指定風格的內容
    styled_prompt = f"[風格: {style_tag}]\n{prompt}"

    # 將 styled_prompt 編碼成模型可接受的 token ID，並轉成 tensor
    input_ids = tokenizer.encode(styled_prompt, return_tensors="pt").to(device)

    # 建立 attention mask，避免模型注意到 padding 的位置（這裡預設沒有 padding，但為保險起見仍處理）
    attention_mask = (input_ids != tokenizer.pad_token_id).long()

    # 使用模型產生文字
    output_ids = model.generate(
        input_ids,                      # 輸入的 token
        attention_mask=attention_mask, # 注意力遮罩，讓模型知道哪些位置是有效輸入
        max_new_tokens=max_new_tokens, # 生成的最大 token 數量
        temperature=temperature,       # 溫度值控制隨機性，越高越多樣化
        top_k=top_k,                   # top-k 篩選（僅在前 k 個最可能的詞中選擇）
        top_p=top_p,                   # top-p (nucleus sampling)，保留機率總和為 p 的詞集合中選擇
        do_sample=True,                # 啟用隨機取樣（非貪婪解碼）
        pad_token_id=tokenizer.pad_token_id
    )

    # 解碼模型輸出的 token ID，轉回可讀文字，並移除多餘空白
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).replace(" ", "")
    return output_text  # 回傳最終生成的文字

In [9]:
prompt_text = "故事開始於一個神秘的森林，小狐狸出現在月光下。"
style = "fairy_tale"
# 呼叫前面定義的 generate_text 函式，傳入提示文字與風格，產生對應風格的續寫文字
result = generate_text(prompt_text, style=style)
print("輸入提示：", prompt_text)
print("風格：", style)
print("產生的故事：", result)

輸入提示： 故事開始於一個神秘的森林，小狐狸出現在月光下。
風格： fairy_tale
產生的故事： [風格:童話]故事開始於一個神秘的森林，小狐狸出現在月光下。於森林裡面存在著一個叫的神祕女子，就在那裡，由於她的存在，因此她們就一直留在森林裡，但這個神祕女子就在那裡，她們的存在便已經破滅。他們在森林裡的小狐狸便逐漸接近於森林的下方，並且沒有足夠的野生動物。在這個時候，森林裡被看上去是獨木舟，一直到她們的存在才一直存在，這個時期裡，當她們的存在便已經被破壞


In [10]:
prompt_text = "故事開始於一個神秘的森林，小狐狸出現在月光下。"
style = "sci_fi"

result = generate_text(prompt_text, style=style)
print("輸入提示：", prompt_text)
print("風格：", style)
print("產生的故事：", result)

輸入提示： 故事開始於一個神秘的森林，小狐狸出現在月光下。
風格： sci_fi
產生的故事： [風格:科幻]故事開始於一個神秘的森林，小狐狸出現在月光下。然牠們因為受到傷害，但小狐狸一直在地下工作，在黑暗中生存，最後只剩下一個種族。這個階段是發生在阿靈頓的天王之前。牠們是一個神祕的星團，由一個不相信的人發射，成為它們的一部分。牠們的祖先是在古代到底是神祕的，所以它們在後來被稱為小狐狸。然而在這些天王之前，有些人認為它們是一個小狐狸。在這個階段之前，小


In [11]:
prompt_text = "故事開始於一個神秘的森林，小狐狸出現在月光下。"
style = "news"

result = generate_text(prompt_text, style=style)
print("輸入提示：", prompt_text)
print("風格：", style)
print("產生的故事：", result)

輸入提示： 故事開始於一個神秘的森林，小狐狸出現在月光下。
風格： news
產生的故事： [風格:新聞]故事開始於一個神秘的森林，小狐狸出現在月光下。種情況非常特別，有時候會變得非常敏感。當森林被大量吸收時，大家應該利用一些更好的表情去對待，一邊為人類提供一些特殊的生物。然而，大多數人還是不會有能力去提高森林的溫度。不過，對於某些細菌學家而言，這些細菌會帶來更多的副作用。大量的細菌會引起細菌的自然刺激，但這些細菌可以促使它們產生抗體，這是一種有效


In [13]:
prompt_text = "故事開始於一個神秘的森林，小狐狸出現在月光下。"
style = "wuxia"

result = generate_text(prompt_text, style=style)
print("輸入提示：", prompt_text)
print("風格：", style)
print("產生的故事：", result)

輸入提示： 故事開始於一個神秘的森林，小狐狸出現在月光下。
風格： wuxia
產生的故事： [風格:武俠]故事開始於一個神秘的森林，小狐狸出現在月光下。事中，小狐狸在美國的漫畫部門工作；而在一次世界大戰中，他與另一名年輕的阿爾弗雷德·霍納德以及一名朋友於德國的一個小村落的居住。由於當地的人民生活環境，他也會為人服務。當地的小狐狸被歸類於風格：中村（），原本稱為中村（），後改名為中村。自古以來，中村有個小村，但後來在東京帝國大學的天文學家伊藤博
